In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import config
from datasets.datasethandler import DatasetHandler
datasetHandler = DatasetHandler()

# Tutorial: Hyperparameter Search

In [3]:
import torch
import pytorch_lightning as pl
import optuna
from optuna.integration import PyTorchLightningPruningCallback

from classification.models.M5 import M5PLModule
from classification.models.SpectrogramCNN import SpectrogramCNNPLModule
from classification.models.DeepRecursiveCNN import DeepRecursiveCNNPLModule

from classification.trainer.HyperParamSearch import MetricsCallback, save_model

In [4]:
from pytorch_lightning.callbacks import Callback

class SaveBestCallback(Callback): 
    def __init__(self, model_name = "newest_model", add_v_number = True):
        super().__init__()
        self.model_name = model_name
        self.best_val_acc = None
        self.add_v_number = add_v_number

    def on_epoch_end(self, trainer, pl_module):
        if not self.best_val_acc or pl_module.val_results_history[-1]["val_acc"] > self.best_val_acc:
            print("new best val acc", pl_module.val_results_history[-1]["val_acc"])
            self.best_val_acc = pl_module.val_results_history[-1]["val_acc"]
            #save_path = self.model_name + (( "_v{}".format(trainer.logger.version) +  "_best.p") if self.add_v_number else "")
            save_path = self.model_name + str(pl_module.val_results_history[-1]["val_acc"]) + ".p"
            pl_module.save(save_path, overwrite_if_exists=True)
            print("Saved checkpoint at epoch {} at \"{}\"".format((trainer.current_epoch + 1), save_path))

cb = SaveBestCallback("optuna_deepRecursive", add_v_number = False)

### Objective

In [5]:
def objective(trial):
    metrics_callback = MetricsCallback()  
        
    # here we sample the hyper params, similar as in our old random search
    trial_hparams = {"batch_size": 5,
                     "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 1e-1),
                     "p_drop": trial.suggest_float("p_drop", 0, 1),
                     "lr_decay": trial.suggest_float("lr_decay", 0.7, 1),
                     "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-1)
                    }    

    model = DeepRecursiveCNNPLModule(trial_hparams)
    
    # create a trainer
    trainer = pl.Trainer(
        logger=pl.loggers.TensorBoardLogger(config.LOG_DIR, name=type(model.model).__name__),
        max_epochs=12,                                                               
        gpus=1 if torch.cuda.is_available() else None,
        callbacks=[metrics_callback, cb],                                                   # save latest accuracy
        early_stop_callback=PyTorchLightningPruningCallback(trial, monitor="validation_acc"), # early stopping
    )
    
    datasetHandler.load(model, 'training')
    datasetHandler.load(model, 'validation')
    trainer.fit(model)

    # save model
    model.save("saved_modelsq/"+'{}.p'.format(trial.number))

    # return validation accuracy from latest model, as that's what we want to minimize by our hyper param search
    return metrics_callback.metrics[-1]["validation_acc"]

pruner = optuna.pruners.MedianPruner()
study = optuna.create_study(direction="maximize", pruner=pruner, study_name='DeepRecursive_study', storage='sqlite:///DeepRecursive_study.db', load_if_exists=True)

[I 2020-07-11 11:29:20,403] Using an existing study with name 'DeepRecursive_study' instead of creating a new one.


### Run Search

In [6]:
study.optimize(objective, n_trials=1000, timeout=1.5*21600) #6h

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]


Loading cached training data of dataset 0 from /nfs/students/summer-term-2020/project-4/data/dataset1/dataset_48k/
Loading cached validation data of dataset 0 from /nfs/students/summer-term-2020/project-4/data/dataset1/dataset_48k/


/nfs/students/summer-term-2020/project-4/yan/.conda/envs/cnn_project/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
Set SLURM handle signals.

   | Name        | Type             | Params
---------------------------------------------
0  | model       | DeepRecursiveCNN | 2 M   
1  | model.bn0   | BatchNorm1d      | 2     
2  | model.conv1 | Conv1d           | 10 K  
3  | model.bn1   | BatchNorm1d      | 256   
4  | model.pool1 | MaxPool1d        | 0     
5  | model.drop1 | Dropout          | 0     
6  | model.conv2 | Conv1d           | 98 K  
7  | model.bn2   | BatchNorm1d      | 512   
8  | model.pool2 | MaxPool1d        | 0     
9  | model.conv3 | Conv1d           | 393 K 
10 | model.bn3   | BatchNorm1d      | 1 K   
11 | model.pool3 | MaxPool1d        | 0

Val-Acc=0.0035566093657379964


/nfs/students/summer-term-2020/project-4/yan/.conda/envs/cnn_project/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


...

Val-Acc=0.004149377593360996


Detected KeyboardInterrupt, attempting graceful shutdown...



Saved model to "saved_modelsq/37.p"


[W 2020-07-11 18:00:53,756] Setting status of trial#37 as TrialState.FAIL because of the following error: IndexError('list index out of range')
Traceback (most recent call last):
  File "/nfs/students/summer-term-2020/project-4/yan/.conda/envs/cnn_project/lib/python3.8/site-packages/optuna/study.py", line 734, in _run_trial
    result = func(trial)
  File "<ipython-input-5-74f73ef680a0>", line 31, in objective
    return metrics_callback.metrics[-1]["validation_acc"]
IndexError: list index out of range


IndexError: list index out of range

In [ ]:
#!mkdir saved_modelsq

In [7]:
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
best_trial = study.best_trial
print("  Value: {}".format(best_trial.value))
print("  Params: ")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 38
Best trial:
  Value: 0.8725548310610551
  Params: 
    learning_rate: 3.387406765841386e-05
    lr_decay: 0.8971336597031795
    p_drop: 0.19001011261775047
    weight_decay: 0.006698692076940405
